In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wikitext2-data/test.txt
/kaggle/input/wikitext2-data/train.txt


# we need to normalize the text data (learning -->Learning both should be same)

In [65]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [66]:
def normalization(data):
    return [line.lower().strip() for line in data if isinstance(line, str)]


# we need to convert the words into the tokens like "i am learning deep learning" --> ["i","am","learning","deep","learning"]

In [67]:
# def tokeinazation(data):
#     return data.split()
# i am using the huggnyface transformer so no need to manually to id it , will handle insode the it seld

# Now need to indexing the words so our model can learn

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
MAX_LEN = 64  # cap sequence length to keep batches memory friendly
def indexing(text_list):
    """
    text_list: List[str]
    returns: token_ids (Tensor), attention_mask (Tensor)
    """
    encoded = tokenizer(
        text_list,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    return encoded["input_ids"], encoded["attention_mask"]

In [69]:
# making the input data and the target data 
# we are divding the data into the input and the target 
# i am learning ---> data we have , now from this we are making the input like 
# if we put 'i' then the model should predict 'am' , like this , so for this we are mapping 
# the data sets , i -> am and more:
def create_sequence(input_ids):
    """
    input_ids: Tensor of shape (batch_size, seq_len)

    Returns:
        inputs  : (batch_size, seq_len-1)
        targets : (batch_size, seq_len-1)
    """
    inputs = input_ids[:, :-1]
    targets = input_ids[:, 1:]
    return inputs, targets

            

# All the prepocessing data

In [ ]:
# load data
data = pd.read_csv('/kaggle/input/wikitext2-data/train.txt', header=None)
data = data[0].dropna().tolist()

# normalization
data = normalization(data)

# indexing (GPT-2 tokenizer)
input_ids, attention_mask = indexing(data)
print(f"Tokenized dataset shape: {input_ids.shape}")
print("Data tensors remain on CPU; move batches to GPU during training.")

passed it
passed it


In [71]:

# from torch.nn.utils.rnn import pad_sequence

# inputs = [torch.tensor(seq) for seq in inputs]
# targets = torch.tensor(targets)
# padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)

In [ ]:
from torch.utils.data import Dataset

class LanguageModelingDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        ids = self.input_ids[idx]
        mask = self.attention_mask[idx]
        inputs = ids[:-1]
        targets = ids[1:]
        tgt_mask = mask[1:].to(torch.float32)
        return inputs, targets, tgt_mask

print("LanguageModelingDataset class defined.")

In [ ]:
from torch.utils.data import DataLoader

dataset = LanguageModelingDataset(
    input_ids=input_ids,
    attention_mask=attention_mask
)

BATCH_SIZE = 16  # start small to keep GPU usage predictable
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=(device.type == "cuda")
)

print(f"Dataset created with {len(dataset)} samples.")
print(f"DataLoader created with batch size {BATCH_SIZE}.")

In [72]:
embedding = torch.nn.Embedding(
    num_embeddings=tokenizer.vocab_size,
    embedding_dim=128,
    padding_idx=tokenizer.pad_token_id
)
print("passed it")


passed it


In [73]:
class LSTMModel(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, pad_token_id):
        super().__init__()

        self.embedding = torch.nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim,
            padding_idx=pad_token_id
        )

        self.lstm = torch.nn.LSTM(
            embed_dim,
            hidden_dim,
            batch_first=True
        )

        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_ids):
        """
        input_ids: (batch_size, seq_len)
        returns logits: (batch_size, seq_len, vocab_size)
        """

        emb = self.embedding(input_ids)          # (B, T, E)
        lstm_out, _ = self.lstm(emb)              # (B, T, H)
        logits = self.fc(lstm_out)                # (B, T, V)

        return logits


In [74]:
model = LSTMModel(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_dim=256,
    pad_token_id=tokenizer.pad_token_id
).to(device)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

In [ ]:
batch_inputs, batch_targets, batch_mask = next(iter(dataloader))
batch_inputs = batch_inputs.to(device)
with torch.no_grad():
    logits = model(batch_inputs)
print("Forward pass on a single batch completed.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.43 GiB is free. Process 4067 has 13.31 GiB memory in use. Of the allocated memory 12.34 GiB is allocated by PyTorch, and 870.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
EPOCHS = 1
model.train()
for epoch in range(EPOCHS):
    total_loss = 0.0
    token_count = 0.0
    for batch_inputs, batch_targets, batch_mask in dataloader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        batch_mask = batch_mask.to(device)
        valid_tokens = batch_mask.sum()
        if valid_tokens == 0:
            continue  # skip empty sequences
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            logits = model(batch_inputs)
            vocab_size = logits.size(-1)
            per_token_loss = loss_fn(
                logits.view(-1, vocab_size),
                batch_targets.view(-1)
            )
            loss = (per_token_loss * batch_mask.view(-1)).sum() / valid_tokens
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * valid_tokens.item()
        token_count += valid_tokens.item()
    epoch_loss = total_loss / max(token_count, 1.0)
    print(f"Epoch {epoch + 1}: loss={epoch_loss:.4f}")
logits = logits.detach()

In [ ]:
with torch.no_grad():
    # logits shape: (B, T, V)
    # take the last timestep of the first example
    last_logits = logits[0, -1]   # (V)

    probs = torch.softmax(last_logits, dim=-1)
    top_probs, top_ids = torch.topk(probs, k=5)

    for prob, token_id in zip(top_probs, top_ids):
        token = tokenizer.decode([token_id.item()])
        print(f"{token!r}  ->  {prob.item():.4f}")
